## **Recognizing Textureless Images**
This notebook will be used to create a metadata file to pinpoint textureless images from the previous results. The output will be a csv file that can be used to discard the images.

In [ ]:
import numpy as np
import pandas as pd
import cv2
from scipy.signal import find_peaks, hilbert
from tqdm import tqdm
from scipy.stats import cauchy
from scipy.special import kl_div
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from PIL import Image


import math
import os

import reconstruction_deep_network
from reconstruction_deep_network.preprocessing.feature_points import FeaturePointDetector
from reconstruction_deep_network.preprocessing.optical_flow import FeatureMatching
from reconstruction_deep_network.data_loader.matterport import MatterPortData

In [ ]:
module_dir = reconstruction_deep_network.__path__[0]
root_dir = os.path.dirname(module_dir)
result_dir = os.path.join(module_dir, "results")
data_dir = os.path.join(root_dir, "data", "v1", "scans")
color_images = os.path.join(data_dir, "17DRP5sb8fy", "matterport_color_images")
metadata_dir = os.path.join(module_dir, "metadata")

In [ ]:
def histogram_analysis(img: np.ndarray):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    histogram = cv2.calcHist([gray_img], [0], None, [256], [0, 256])
    histogram = histogram.squeeze()
    normalized_hist = ((histogram - np.min(histogram)) / (np.max(histogram) - np.min(histogram)))
    variance = np.var(normalized_hist)
    return normalized_hist

def count_peaks(histogram: np.ndarray):
    peaks = find_peaks(histogram, width=2)
    peak_indices = peaks[0]
    return peak_indices

def model_cauchy_distribution(histogram: np.ndarray):
    x_mean = np.sum(np.arange(len(histogram)) * histogram) / np.sum(histogram)
    scale_parameter = 20  # You can adjust this value
    x = np.linspace(0, 256, len(histogram))
    pdf_cauchy = cauchy.pdf(x, loc=x_mean, scale=scale_parameter)    

    # Calculate the PDF of the Student's t-distribution
    pdf_cauchy = np.clip(pdf_cauchy, 0, 256)
    pdf_cauchy /= np.sum(pdf_cauchy)
    return pdf_cauchy

def compute_kl_divergence(hist_simulated, hist):
    epsilon = 1e-5
    divergence = np.sum(kl_div(hist_simulated + epsilon, hist + epsilon))
    return divergence

def compute_envelope(x_signal: np.ndarray):
    x_hilbert = hilbert(x_signal)
    envelope = np.abs(x_hilbert)
    return envelope

def hl_envelopes_idx(s, dmin=1, dmax=1, split=False):
    """
    Input :
    s: 1d-array, data signal from which to extract high and low envelopes
    dmin, dmax: int, optional, size of chunks, use this if the size of the input signal is too big
    split: bool, optional, if True, split the signal in half along its mean, might help to generate the envelope in some cases
    Output :
    lmin,lmax : high/low envelope idx of input signal s
    """

    # locals min      
    lmin = (np.diff(np.sign(np.diff(s))) > 0).nonzero()[0] + 1 
    # locals max
    lmax = (np.diff(np.sign(np.diff(s))) < 0).nonzero()[0] + 1 
    
    if split:
        # s_mid is zero if s centered around x-axis or more generally mean of signal
        s_mid = np.mean(s) 
        # pre-sorting of locals min based on relative position with respect to s_mid 
        lmin = lmin[s[lmin]<s_mid]
        # pre-sorting of local max based on relative position with respect to s_mid 
        lmax = lmax[s[lmax]>s_mid]

    # global min of dmin-chunks of locals min 
    lmin = lmin[[i+np.argmin(s[lmin[i:i+dmin]]) for i in range(0,len(lmin),dmin)]]
    # global max of dmax-chunks of locals max 
    lmax = lmax[[i+np.argmax(s[lmax[i:i+dmax]]) for i in range(0,len(lmax),dmax)]]
    
    return lmin,lmax


In [ ]:
data_loader = MatterPortData("17DRP5sb8fy")

In [ ]:
all_images = os.listdir(color_images)
color_images = list(filter(lambda x: x.split(".")[-1] == "jpg", all_images))


In [ ]:
feature_metadata_df = pd.DataFrame()

for file_name in tqdm((color_images)):

    img_name = file_name.split(".")[0]

    panorama_id, cam_index, yaw_index = img_name.split("_")
    cam_index = cam_index[1:]

    color_image = data_loader.load_color_image(panorama_id, int(cam_index), int(yaw_index))

    pixel_hist = histogram_analysis(color_image)

    peak_indices = count_peaks(pixel_hist)

    simulated_hist = model_cauchy_distribution(pixel_hist)

    divergence = compute_kl_divergence(simulated_hist, pixel_hist)

    metadata_dict = {"panorama_id": panorama_id,
                     "cam_index": cam_index,
                     "yaw_index": yaw_index,
                     "histogram_peaks": len(peak_indices),
                     "kl_divergence": divergence,
                     }
    

    feature_metadata_df = pd.concat([feature_metadata_df, pd.DataFrame([metadata_dict])], ignore_index=True)



In [ ]:
feature_metadata_df.head()

## **Verification**

In [ ]:
## distribution of number of peaks
fig = px.box(feature_metadata_df, y = ["histogram_peaks", "kl_divergence"])
fig.show()

In [ ]:
## correlation between features
fig = px.scatter(feature_metadata_df, x = "kl_divergence", y = "histogram_peaks")
fig.show()

In [ ]:
category_metatada = os.path.join(metadata_dir, "metadata_final2.csv")
category_metatada_df = pd.read_csv(category_metatada)
column_names = list(category_metatada_df.columns)
category_metatada_df["total_prob"] = category_metatada_df[column_names[1:]].sum(axis=1)
category_metatada_df.head()

## **Merge Metadata Files**

In [ ]:
category_metatada_df["panorama_id"] = category_metatada_df["panaromic_id"].apply(lambda x: x.split(".")[0].split("_")[0])
category_metatada_df["cam_index"] = category_metatada_df["panaromic_id"].apply(lambda x: (x.split(".")[0].split("_")[1][-1]))
category_metatada_df["yaw_index"] = category_metatada_df["panaromic_id"].apply(lambda x: x.split(".")[0].split("_")[-1])
category_metatada_df.head()

In [ ]:
merged_df = feature_metadata_df.merge(category_metatada_df, how='inner', on=['panorama_id', 'cam_index', 'yaw_index'])

In [ ]:
merged_df = merged_df.drop("panaromic_id", axis=1)
merged_df.head()

In [ ]:
merged_df["object_detected"] = merged_df["total_prob"].apply(lambda x: "no_object" if x == 0 else "object")
merged_df.head()

In [ ]:
fig = px.box(merged_df, x = "object_detected", y = "histogram_peaks")
fig.show()

In [ ]:
fig = px.box(merged_df, x = "object_detected", y = "kl_divergence")
fig.show()

Conclusion: The object detection method has not exactly helped since the true positive rate is quite high for the prediction model

## **Testing Hypothesis 1**
Hypothesis: If the number of histogram peaks is greater than a threshold we can safely say that the image is not textureless

In [ ]:


threshold = 5

def change_object_class_based_on_hist(row):
    if row["total_prob"] == 0:
        if row["histogram_peaks"] > threshold:
            return "object"
        else:
            return "no_object"
    
    return "object"

merged_df["object_detected"] = merged_df.apply(change_object_class_based_on_hist, axis=1)
merged_df

In [ ]:
fig = px.box(merged_df, x = "object_detected", y = "histogram_peaks")
fig.show()

In [ ]:
print(len(merged_df.loc[merged_df["object_detected"] == "no_object"]))

In [ ]:
no_object_df = merged_df.loc[merged_df["object_detected"] == "no_object"]

In [ ]:
if not os.path.isdir(os.path.join(result_dir, "no_objects")):
    os.makedirs(os.path.join(result_dir, "no_objects"))

for idx, row in tqdm(no_object_df.iterrows()):
    panorama_id, cam_index, yaw_index = row["panorama_id"], row["cam_index"], row["yaw_index"]
    file_name = f"{panorama_id}_i{cam_index}_{yaw_index}.jpg"

    file_prefix_path = os.path.join(color_images, file_name)
    image = Image.open(file_prefix_path)

    updated_path = os.path.join(result_dir, "no_objects", file_name)
    image = image.save(updated_path)

    

In [ ]:
no_object_df[no_object_df["panorama_id"] == "194bef4cbba641dc9860ba7f43a0c16a"]

In [ ]:

fig = px.violin(no_object_df, y="kl_divergence", box=True, # draw box plot inside the violin
                points='all', # can be 'outliers', or False
               )
fig.show()

In [ ]:
kl_divergence_thr = 80.0
no_object_finetuned = no_object_df.loc[no_object_df["kl_divergence"] <= kl_divergence_thr]

In [ ]:
if not os.path.isdir(os.path.join(result_dir, "no_objects_finetuned")):
    os.makedirs(os.path.join(result_dir, "no_objects_finetuned"))

for idx, row in tqdm(no_object_finetuned.iterrows()):
    panorama_id, cam_index, yaw_index = row["panorama_id"], row["cam_index"], row["yaw_index"]
    file_name = f"{panorama_id}_i{cam_index}_{yaw_index}.jpg"

    file_prefix_path = os.path.join(color_images, file_name)
    image = Image.open(file_prefix_path)

    updated_path = os.path.join(result_dir, "no_objects_finetuned", file_name)
    image = image.save(updated_path)

## **Save Metadata Files**

In [ ]:
merged_df.to_csv(os.path.join(metadata_dir, "feature_metadata.csv"), index=False)
no_object_df.to_csv(os.path.join(metadata_dir, "no_objects.csv"), index=False)